## Import the required **libraries**

In [ ]:
!pip install transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import xlrd as xl
from pandas import ExcelWriter
from pandas import ExcelFile
import pickle, re, json, os, datetime, time

import pprint
pp = pprint.PrettyPrinter(indent=4)
import numpy as np
import plotly.graph_objects as go
import torch
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
print("GPU:", tf.config.list_physical_devices('GPU'))
print("Num GPUs:", len(physical_devices))

GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Num GPUs: 1
GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Num GPUs: 1


In [ ]:
# torch.version.cuda
# torch.cuda.get_arch_list()
# torch.cuda.device_count()
# torch.cuda.current_device()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Read the Data

#### Deep Learning Using BERT  Model####

In [ ]:
!nvidia-smi

Sun Jun  4 08:34:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    11W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

***Our project data is collected from OSS dataset. ***

The dataset contains 9737 records and (text contents, label)attributes, and have been manually classified into 13 categories (i.e. labels).

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/ML_Project_0V3/oversampleddata.csv')
data['Text Content']=data['Text Content'].astype(str)
data['Code']=data['Code'].astype(str)

In [ ]:
data['Code'].value_counts()

Expected Behaviour                   749
Motivation                           749
Social Conversation                  749
Contribution and Commitment          749
Task Progress                        749
Workarounds                          749
Action on Issue                      749
Potential New Issues and Requests    749
Usage                                749
Observed Bug Behaviour               749
Investigation and Exploration        749
Bug Reproduction                     749
Social Convesation                   749
Name: Code, dtype: int64

Expected Behaviour                   749
Motivation                           749
Social Conversation                  749
Contribution and Commitment          749
Task Progress                        749
Workarounds                          749
Action on Issue                      749
Potential New Issues and Requests    749
Usage                                749
Observed Bug Behaviour               749
Investigation and Exploration        749
Bug Reproduction                     749
Social Convesation                   749
Name: Code, dtype: int64

In [ ]:
# Check column name
data.columns

Index(['Unnamed: 0', 'Text Content', 'Code'], dtype='object')

Index(['Unnamed: 0', 'Text Content', 'Code'], dtype='object')

In [ ]:
# Drop "Unname" column 

data = data.loc[:, ~data.columns.str.contains('^Unnamed')]

In [ ]:
#Check column name again
data.columns

Index(['Text Content', 'Code'], dtype='object')

Index(['Text Content', 'Code'], dtype='object')

In [ ]:
# Check the summary

data.describe()

,Text Content,Code
count,9737,9737
unique,4358,13
top,working code,Expected Behaviour
freq,84,749


,Text Content,Code
count,9737,9737
unique,4358,13
top,working code,Expected Behaviour
freq,84,749


## Encoding the labels

### Before encoding the labels, we ***converted label value into the corresponding number(i.e .0,1,2,...)***


In [ ]:
possible_labels = data.Code.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'Expected Behaviour': 0,
 'Motivation': 1,
 'Social Conversation': 2,
 'Contribution and Commitment': 3,
 'Task Progress': 4,
 'Workarounds': 5,
 'Action on Issue': 6,
 'Potential New Issues and Requests': 7,
 'Usage': 8,
 'Observed Bug Behaviour': 9,
 'Investigation and Exploration': 10,
 'Bug Reproduction': 11,
 'Social Convesation': 12}

{'Expected Behaviour': 0,
 'Motivation': 1,
 'Social Conversation': 2,
 'Contribution and Commitment': 3,
 'Task Progress': 4,
 'Workarounds': 5,
 'Action on Issue': 6,
 'Potential New Issues and Requests': 7,
 'Usage': 8,
 'Observed Bug Behaviour': 9,
 'Investigation and Exploration': 10,
 'Bug Reproduction': 11,
 'Social Convesation': 12}

In [ ]:
data['label'] = data.Code.replace(label_dict)   

In [ ]:
data['label']

0       0
1       1
2       2
3       2
4       1
       ..
9732    5
9733    5
9734    5
9735    5
9736    5
Name: label, Length: 9737, dtype: int64

0       0
1       1
2       2
3       2
4       1
       ..
9732    5
9733    5
9734    5
9735    5
9736    5
Name: label, Length: 9737, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(data.index.values, 
                                                  data.label.values, 
                                                  test_size=0.2, 
                                                  random_state=42, 
                                                  stratify=data.label.values)

data['data_type'] = ['not_set']*data.shape[0]

data.loc[X_train, 'data_type'] = 'train'
data.loc[X_val, 'data_type'] = 'val'

data.groupby(['Text Content', 'label', 'data_type']).count()

Code
Text Content                                       label data_type      
\nREFERENCE                                        2     train         2
                                                   10    train         4
![captura de pantalla de 2018-05-25 17-53-11] URL  9     train         2
![captura de pantalla de 2018-05-25 17-54-59] URL  9     train         2
                                                         val           1
...                                                                  ...
ð                                                  2     train         6
                                                         val           1
ð Guys please before commenting +1 or +whatever... 2     train         1
ðPlease !                                          2     train         1
ð¯ ð                                               2     train         1

[5348 rows x 1 columns]

Code
Text Content                                       label data_type      
\nREFERENCE                                        2     train         2
                                                   10    train         4
![captura de pantalla de 2018-05-25 17-53-11] URL  9     train         2
![captura de pantalla de 2018-05-25 17-54-59] URL  9     train         2
                                                         val           1
...                                                                  ...
ð                                                  2     train         6
                                                         val           1
ð Guys please before commenting +1 or +whatever... 2     train         1
ðPlease !                                          2     train         1
ð¯ ð                                               2     train         1

[5348 rows x 1 columns]

### **The distribution of Labels with their count**


*** BertTokenizer and Encoding the Data***

**Tokenization** is a process to take raw texts and split into tokens, which are numeric data to represent words.

Constructs a ***BERT tokenizer***. Based on WordPiece.
Instantiate a ***pre-trained BERT model configuration*** to encode our data.
To convert all the titles from text into encoded form, we use a function called batch_encode_plus , and we will proceed train and validation data separately.
The 1st parameter inside the above function is the title text.
***add_special_tokens=True*** means the sequences will be encoded with the special tokens relative to their model.
When batching sequences together, we set ***return_attention_mask=True***, so it will return the attention mask according to the specific tokenizer defined by the ***max_length*** attribute.
We also want to pad all the titles to certain maximum length.
We actually do not need to set **max_length=512**, but just to play it safe.
***return_tensors='pt'*** to return PyTorch.
And then we need to split the data into ***input_ids***, ***attention_masks*** and ***labels***.
Finally, after we get encoded data set, we can create training data and validation data.

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    data[data['data_type']=='train']['Text Content'].values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    data[data['data_type']=='val']['Text Content'].values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(data[data['data_type']=='train']['label'].values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(data[data['data_type']=='val']['label'].values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `

### BERT Pre-trained **Model**

We are working each text context as its unique sequence, so one sequence will be classified to one of the ***13 labels*** (i.e. Expected Behaviour, Motivation and Social Conversation, etc.).

***bert-base-uncased*** is a smaller pre-trained model.
Using num_labels to indicate the number of output labels.
We don’t really care about output_attentions.
We also don’t need output_hidden_states.

---



In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

### Data **Loaders**

DataLoader combines a dataset and a sampler, and provides an iterable over the given dataset.
We use ***RandomSampler*** for training and ***SequentialSampler*** for validation.
Given the limited memory in my environment, we set batch_size=3.

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

### Optimizer & **Scheduler**

To construct an optimizer, we have to give it an iterable containing the parameters to optimize. Then, we can specify ***optimizer-specific*** options such as the learning rate, epsilon, etc.
I found epochs=10 works well for this data set.
Create a schedule with a learning rate that decreases linearly from the initial learning rate set in the optimizer to 0, after a warmup period during which it increases linearly from 0 to the initial learning rate set in the optimizer.

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 7

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


###Performance Metrics

We are going to create three functions which are ***accuracy_score_func***,***f1_ score_func*** and **accuracy_per_class** as performance metrics.

In [ ]:
from sklearn.metrics import f1_score,accuracy_score



def accuracy_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return accuracy_score(labels_flat, preds_flat)

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

### Training **Loop**



In [ ]:
import random

# Set device (CPU or GPU)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda")
model.cuda()
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
device = torch.device("cuda")
model.cuda()
def evaluate(dataloader_val):

    model.eval()
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)  
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy() 
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch) 
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       
        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    val_accuracy_score= accuracy_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')
    tqdm.write(f'Accuracy score: {val_accuracy_score}')

  0%|          | 0/7 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/2597 [00:00<?, ?it/s]


Epoch 1
Training loss: 1.7590523932256812
Validation loss: 1.1268803313173927
F1 Score (Weighted): 0.6498838351006166
Accuracy score: 0.662217659137577


Epoch 2:   0%|          | 0/2597 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.8309606414637574
Validation loss: 0.7376874224103701
F1 Score (Weighted): 0.7585887496620324
Accuracy score: 0.7725872689938398


...


Epoch 5
Training loss: 0.15549081224858582
Validation loss: 0.8058215399844294
F1 Score (Weighted): 0.8546920049865618
Accuracy score: 0.8608829568788501


Epoch 6:   0%|          | 0/2597 [00:00<?, ?it/s]


Epoch 6
Training loss: 0.11678287116449375
Validation loss: 0.7769179870254839
F1 Score (Weighted): 0.86798698129177
Accuracy score: 0.87217659137577


Epoch 7:   0%|          | 0/2597 [00:00<?, ?it/s]


Epoch 7
Training loss: 0.08177271696448836
Validation loss: 0.7983564020696212
F1 Score (Weighted): 0.8666599646868876
Accuracy score: 0.8711498973305954


***Loading and Evaluating the Model***

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

model.load_state_dict(torch.load('finetuned_BERT_epoch_1.model', map_location=torch.device('cuda')))

_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at